In [202]:
import csv
import os
import re
import pandas as pd
from bs4 import BeautifulSoup
# エンコーディング自動検出用
from charset_normalizer import from_path

In [203]:
# HTMLファイルが格納されているディレクトリのパス
directory = "www.geocities.co.jp/Playtown-Dice/6159"

In [204]:
def is_mojibake(text):
    """
    簡易的な文字化け検出関数。非表示文字や無効な文字がある場合にTrueを返す。
    """
    # 文字化けによく見られる文字パターンを除外（ここでは基本的な制御文字を例に）
    mojibake_patterns = [
        r'[�]'  # 置換文字
    ]
    
    for pattern in mojibake_patterns:
        if re.search(pattern, text):
            return True
    return False

In [205]:
def read_html_with_fallback(file_path: str) -> BeautifulSoup:
    # 優先的に Shift_JIS / CP932 を試す
    try:
        with open(file_path, "r", encoding="shift_jis") as f:
            content = f.read()
        return BeautifulSoup(content, "html.parser")
    except UnicodeDecodeError:
        pass  # Shift_JIS 読み込みに失敗した場合は次へ

    try:
        with open(file_path, "r", encoding="cp932") as f:
            content = f.read()
        return BeautifulSoup(content, "html.parser")
    except UnicodeDecodeError:
        pass  # CP932 でも失敗した場合は自動判定へ

    # charset_normalizer による自動検出
    result = from_path(file_path).best()
    if result is None:
        raise ValueError(f"エンコーディングの検出に失敗しました: {file_path}")

    content = result.decoded
    return BeautifulSoup(content, "html.parser")

In [206]:
def extract_all_entries_from_html(file_path):
    """
    HTML内のすべての trペア から {title, issue_info, body_text} を抽出する
    """
    try:
        soup = read_html_with_fallback(file_path)

        rows = soup.find_all("tr")
        entries = []

        # ペアで処理（title/infoが1行目, 本文が2行目）
        for i in range(0, len(rows) - 1, 2):
            tr_title = rows[i]
            tr_body = rows[i + 1]

            # タイトル
            title_tag = tr_title.find("font", attrs={"size": "+2"})
            title = title_tag.get_text(strip=True) if title_tag else None

            # 出典
            issue_tag = tr_title.find("font", attrs={"size": "-1"})
            issue_info = issue_tag.get_text(strip=True) if issue_tag else None

            # 本文（1つまたは複数）
            # span.line タグが壊れてる可能性に備え、tr_body 全体を文字列として処理
            raw_html = str(tr_body)

            # <span class="line">〜(次の <span> or </tr> or <td> まで)
            raw_spans = re.findall(r'<span class="line">(.*?)(?=<span class="line">|<td bgcolor="#80ffff">|</tr>|</td>)', raw_html, re.DOTALL)
            # HTMLタグを除去して本文だけにする
            body_texts = [BeautifulSoup(fragment, "html.parser").get_text(separator=" ", strip=True) for fragment in raw_spans]

            # 最初の要素がお目当ての本文なので取得
            body_text = body_texts[0]

            entries.append({
                "title": title,
                "issue_info": issue_info,
                "body_text": body_text
            })

        return entries

    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return []

In [207]:
def extract_number_from_filename(file: str) -> int | None:
    match = re.match(r"^d-(\d+)\.html?$", file)
    return int(match.group(1)) if match else None

In [208]:
def extract_d_xx_recursive_to_df(directory, max_pages=1000):
    """
    指定されたディレクトリを再帰的に検索し、HTMLファイルからテキストを抽出し、
    ディレクトリ構造をキーとしてDataFrameに格納する関数
    """
    data = []
    page_count = 0
    
    for root, _dirs, files in os.walk(directory):
        for file in files:
            match_index = extract_number_from_filename(file)
            if match_index:
                file_path = os.path.join(root, file)
                stories = extract_all_entries_from_html(file_path)
                if len(stories) == 0: continue
                # ディレクトリ構造と抽出したテキストをDataFrameに格納するためリストに追加
                for index, story in enumerate(stories, 1):
                    data.append({
                        "directory": root,
                        # "dirs": _dirs,
                        "file": file,
                        "volume": match_index,
                        "story_index": index,
                        "title": story["title"],
                        "issue_info": story["issue_info"],
                        "body_text": story["body_text"]
                    })
                    
                page_count += 1
                if page_count >= max_pages:
                    print(f"Processed {max_pages} pages. Stopping...")
                    return pd.DataFrame(data)  # DataFrameに変換して返す
    
    return pd.DataFrame(data)

In [209]:
# 実行
df = extract_d_xx_recursive_to_df(directory)

In [210]:
df.sort_values(by=["volume", "story_index"], inplace=True, ignore_index=True)
# データフレームを表示
# print(df.head())

In [211]:
df.to_csv("doraemon_stories.csv", index=False, escapechar='\\', quoting=csv.QUOTE_ALL)